In [ ]:
# Youtube Link: https://www.youtube.com/watch?v=PgLjwl6Br0k
# this is the main tkinter file
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# initalise the tkinter GUI
root = tk.Tk()

root.geometry("1200x600") # set the root dimensions
root.pack_propagate(False) # tells the root to not let the widgets inside it determine its size.
root.resizable(0, 0) # makes the root window fixed in size.

# Frame for TreeView
frame1 = tk.LabelFrame(root, text="Dataset")
frame1.place(height=350, width=600)

#Frame fro data.describe()
# frame3 = tk.LabelFrame(root, text="Data Description")
# frame3.place(height=150, width=600, rely = 0.7, relx = 0.5)
frame3 = tk.LabelFrame(root)
frame3.place(height=150, width=600, rely = 0.7, relx = 0.5)


# Frame for data visualization
frame2 = tk.LabelFrame(root, text ='Data visualization')
frame2.place(height =400, width = 600, rely = 0, relx = 0.5)

# Frame for open file dialog
file_frame = tk.LabelFrame(root, text="Open File")
file_frame.place(height=100, width=500, rely=0.65, relx=0)

# Buttons
button1 = tk.Button(file_frame, text="Browse A File", command=lambda: File_dialog())
button1.place(rely=0.65, relx=0.70)

button2 = tk.Button(file_frame, text="Load File", command=lambda: Load_excel_data())
button2.place(rely=0.65, relx=0.50)

button3 = tk.Button(file_frame, text="clean data", command=lambda: remove_fill_na())
button3.place(rely=0.65, relx=0.30)

button4 = tk.Button(file_frame, text="visualize data", command=lambda: visualization())
button4.place(rely=0.65, relx=0.10)

# The file/file path text
label_file = ttk.Label(file_frame, text="No File Selected")
label_file.place(rely=0, relx=0)


## Treeview Widget for dataset
tv1 = ttk.Treeview(frame1)
tv1.place(relheight=1, relwidth=1) # set the height and width of the widget to 100% of its container (frame1).

treescrolly = tk.Scrollbar(frame1, orient="vertical", command=tv1.yview) # command means update the yaxis view of the widget
treescrollx = tk.Scrollbar(frame1, orient="horizontal", command=tv1.xview) # command means update the xaxis view of the widget
tv1.configure(xscrollcommand=treescrollx.set, yscrollcommand=treescrolly.set) # assign the scrollbars to the Treeview Widget
treescrollx.pack(side="bottom", fill="x") # make the scrollbar fill the x axis of the Treeview widget
treescrolly.pack(side="right", fill="y") # make the scrollbar fill the y axis of the Treeview widget

#treeview for visualization
# tv2 = ttk.Treeview(frame2)
# tv2.place(relheight=1, relwidth=1) # set the height and width of the widget to 100% of its container (frame1).

# treescrolly = tk.Scrollbar(frame2, orient="vertical", command=tv2.yview) # command means update the yaxis view of the widget
# treescrollx = tk.Scrollbar(frame2, orient="horizontal", command=tv2.xview) # command means update the xaxis view of the widget
# tv2.configure(xscrollcommand=treescrollx.set, yscrollcommand=treescrolly.set) # assign the scrollbars to the Treeview Widget
# treescrollx.pack(side="bottom", fill="x") # make the scrollbar fill the x axis of the Treeview widget
# treescrolly.pack(side="right", fill="y") # make the scrollbar fill the y axis of the Treeview widget

#view for data.describe()
# tv3 = ttk.Treeview(frame3)
# tv3.place(relheight=1, relwidth=1) # set the height and width of the widget to 100% of its container (frame1).

# treescrolly = tk.Scrollbar(frame3, orient="vertical", command=tv3.yview) # command means update the yaxis view of the widget
# treescrollx = tk.Scrollbar(frame3, orient="horizontal", command=tv3.xview) # command means update the xaxis view of the widget
# # tv3.configure(xscrollcommand=treescrollx.set, yscrollcommand=treescrolly.set) # assign the scrollbars to the Treeview Widget
# treescrollx.pack(side="bottom", fill="x") # make the scrollbar fill the x axis of the Treeview widget
# treescrolly.pack(side="right", fill="y") # make the scrollbar fill the y axis of the Treeview widget


df = None

def File_dialog():
    """This Function will open the file explorer and assign the chosen file path to label_file"""
    filename = filedialog.askopenfilename(initialdir="/",
                                          title="Select A File",
                                          filetype=(("All Files", "*.*") , ))
    label_file["text"] = filename
    return None


def Load_excel_data():
    """If the file selected is valid this will load the file into the Treeview"""
    global df
    file_path = label_file["text"]
    try:
        excel_filename = r"{}".format(file_path)
        if excel_filename[-4:] == ".csv":
            df = pd.read_csv(excel_filename)
        else:
            df = pd.read_excel(excel_filename)

    except ValueError:
        tk.messagebox.showerror("Information", "The file you have chosen is invalid")
        return None
    except FileNotFoundError:
        tk.messagebox.showerror("Information", f"No such file as {file_path}")
        return None

    clear_data()
    showData(tv1 , df)

def remove_fill_na():
    global df
#     print(df)
    df1=df.dropna(thresh=df.shape[0]*0.4,how='all',axis=1).copy()
    df1=outliers(df1)
    for i in df1.columns:
        if df1[i].dtypes=='object':
            df1[i]=df1[i].fillna(df1[i].mode()[0])
        else:
            df1[i]=df1[i].fillna(df1[i].mean())
    
    clear_data()
    showData(tv1 , df1)

def outliers(data):
    df_numeric = data.select_dtypes(include=[np.number])
    numeric_cols = df_numeric.columns.values
    index_list = []
    for feature in numeric_cols:
        index_list.extend(get_outliers_index(df, feature))
    print(remove(data , index_list).shape)
    return remove(data , index_list)
    
def get_outliers_index(data,ft):
    Q1 = data[ft].quantile(0.25)
    Q3 = data[ft].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    ls = data.index[ (df[ft] < lower_bound) | (df[ft] > upper_bound)]
    return ls

# df_numeric = df.select_dtypes(include=[np.number])
# numeric_cols = df_numeric.columns.values
# index_list = []
# for feature in numeric_cols:
#     index_list.extend(outliers(df, feature))
#remove the outliers

def remove(data,ls):
    ls = sorted(set(ls))
    data = data.drop(ls)
    return data

# this is not workoing. code is good but showing error when run in 
def visualization():
    corelation = df.corr()
    sns.set(style="white")

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(1, 1))
    f = plt.figure(figsize=(1,1))
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corelation, xticklabels = corelation.columns, yticklabels = corelation.columns, annot = True)
    canvas = FigureCanvasTkAgg(f,master=frame2)
#     scrollbary_canv = tk.Scrollbar(frame2 , orient = 'vertical')
#     scrollbary_canv.pack(side = 'right' , fill = 'y')
    
#     scrollbarx_canv = tk.Scrollbar(frame2 , orient = 'horizontal')
#     scrollbarx_canv.pack(side = 'bottom' , fill = 'x')
#     canvas_frame.config(yscrollcommand=scrollbary_canv.set , xscrollcommand = scrollbarx_canv.set)
    
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

def showData(tv , df):
    tv["column"] = list(df.columns)
    tv["show"] = "headings"
    for column in tv["columns"]:
        tv.heading(column, text=column) # let the column heading = column name

    df_rows = df.to_numpy().tolist() # turns the dataframe into a list of lists
    for row in df_rows:
        tv.insert("", "end", values=row) # inserts each list into the treeview. 
        
    colrow = tk.Label(root , text= str(len(df.axes[0])) + "x" + str(len(df.axes[1])))
    colrow.place(x=0 , y =500 , width = 500)
    
    describe = tk.Text(frame3) # I want this in  the data description frame
    describe.insert('end' , df.describe())

    scrollbary_des = tk.Scrollbar(describe , orient = 'vertical')
    scrollbary_des.pack(side = 'right' , fill = 'y')
    describe['yscrollcommand'] = scrollbary_des.set
    describe.place(x=0 , y =0 , height=150, width=600)
    
    
    return None
    
# df1=remove_fill_na(df)

def clear_data():
    tv1.delete(*tv1.get_children())
    return None


root.mainloop()

(344, 16)


In [2]:
# This is he t visualization function, it is working just fine but in tkinter its showing error
#create a general visualization function 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv(r"penguins_lter.csv")

def visualization():
    corelation = df.corr()
    sns.heatmap(corelation, xticklabels = corelation.columns, yticklabels = corelation.columns, annot = True)
    sns.pairplot(df)
    
    df_numeric = df.select_dtypes(include=[np.number])
    numeric_cols = df_numeric.columns.values
    
    for col in numeric_cols:
        sns.displot(df[col], bins = 20, color = 'green')
        sns.histplot(df[col], bins = 20, color = 'yellow')
    
    for col in numeric_cols:
        plt.hist(df[col], bins = np.arange(10,100,5), color = 'purple');
    
visualization()

FileNotFoundError: [Errno 2] No such file or directory: 'penguins_lter.csv'